<a href="https://colab.research.google.com/github/TheAllUnknown/node-feature-supported-LightGCN-for-movie-recommendation/blob/main/Custmoized_LightGCN_for_movie_recommednation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MovieLens recommendation
In this project, we will edvelope a custmoized LightGCN model for movie recommendation. PyG and pytorch will be the main package for this task.

In [1]:
!pip install torch_geometric
!pip install sentence_transformers #needed for MovieLens dataset
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.3/497.3 kB 572.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.7/745.7 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 20.2 MB/s eta 0:00:00


In [2]:
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch_geometric
from torch_geometric import datasets
import torch
from torch import nn, optim, Tensor

import torch.nn.functional as F

/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/libpyg.so: undefined symbol: _ZN2at4_ops10zeros_like4callERKNS_6TensorEN3c108optionalINS5_10ScalarTypeEEENS6_INS5_6LayoutEEENS6_INS5_6DeviceEEENS6_IbEENS6_INS5_12MemoryFormatEEE
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Data Preparation
1. download data first and transfer it from directed heterodata graph to undirected homogeneous graph.
2. split edges to train and test, and make sure the train edge_index graph is connected, i.e. no isolated nodes


In [8]:
MovieLens = datasets.MovieLens(root='data')
# the raw data is just several csv file
# the processed data are in stored in a pickle file

# By default, we will use a senetecne transformer to get the word embedding for the movie genre, this embedding will be movie node features.
# This is done automatically in datasets.MovieLens(root='data')  call
# we can also pass other model to transformer: https://huggingface.co/sentence-transformers

data = MovieLens.data

In [9]:

data.__dict__['_node_store_dict'] = {
    key: data._node_store_dict[key]
    for key in ["user", "movie"]
} # change the order of the node type. when tranferring to Homogeneous, this will be helpful.
print(data._node_store_dict)

{'user': {'num_nodes': 610}, 'movie': {'x': tensor([[-0.0828,  0.0530,  0.0536,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1053,  0.1508, -0.0264,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0988,  0.0177, -0.0527,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.1115,  0.0310, -0.0177,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0366,  0.0137,  0.0315,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0500, -0.0141, -0.0031,  ...,  0.0000,  0.0000,  0.0000]])}}


In [10]:
data._node_store_dict

{'user': {'num_nodes': 610},
 'movie': {'x': tensor([[-0.0828,  0.0530,  0.0536,  ...,  0.0000,  0.0000,  0.0000],
         [-0.1053,  0.1508, -0.0264,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0988,  0.0177, -0.0527,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.1115,  0.0310, -0.0177,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0366,  0.0137,  0.0315,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0500, -0.0141, -0.0031,  ...,  0.0000,  0.0000,  0.0000]])}}

##### HeteroData

hetero data has different types of nodes and edges which makes it quite different from homogeneous data. In genral There are two types of info stored in heterodata obeject: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.HeteroData.html#torch_geometric.data.HeteroData
* node info: can be accessed through `data['node_name']`, and then we can use all method that is suitable for homogeneous data on this data object.
* edge ino: a tuple `(source_node_name,edge_type_name,des_node_name)`, can be accessed through `data[source_node_name,edge_type_name,des_node_name]`

In [11]:
data['user', 'rates', 'movie'].edge_index #
data.stores
data.node_types # return node names list
data.node_stores # node features, here we only have on dict because the users don't have featuers
# so there is no info about users in node_stores
data.num_nodes # 10352 nodes in total
data.num_node_features # 404 features for movie, no feature for user

data.edge_types # return only one type of edge
data.edge_stores # we can tell that the source_node and des_node use different node id systmes, the edge_label is rating

data.node_items() # a list of tuples, here two tuples, movie and user


[('user', {'num_nodes': 610}),
 ('movie',
  {'x': tensor([[-0.0828,  0.0530,  0.0536,  ...,  0.0000,  0.0000,  0.0000],
          [-0.1053,  0.1508, -0.0264,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0988,  0.0177, -0.0527,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [-0.1115,  0.0310, -0.0177,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0366,  0.0137,  0.0315,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0500, -0.0141, -0.0031,  ...,  0.0000,  0.0000,  0.0000]])})]

##### filter the positive feedback
We only keep the edges that have positive feeback（i.e.the edge_label is greater than 3）, and then we add another type of edge in the graph, which is the reverse of orginal edge. This will help us to transform the graph to undirected homogeneous graph.

In [12]:
from torch_geometric.utils import to_undirected, remove_isolated_nodes
from sklearn.model_selection import train_test_split


filter = data[('user','rates','movie')]['edge_label'] >= 3
data[('user','rates','movie')]['edge_index'] = data[('user','rates','movie')]['edge_index'][:,filter] # in the raw data we have 100836 edges, after filter we have 81763 left only

edge_index = data[('user','rates','movie')]['edge_index'][[1,0],:]
# sorted_indices = torch.argsort(edge_index[0, :])

# Use the sorted indices to rearrange the tensor
data[('movie','rated_by','user')]['edge_index'] = edge_index

del data[('user','rates','movie')]['edge_label'] # delete the edge_label since we don't need any more


##### transform to undirected homogeneous

In [13]:
homodata = data.to_homogeneous().to(device)
homodata.node_type


tensor([0, 0, 0,  ..., 1, 1, 1], device='cuda:0')

In [14]:
print('the total number of node is:',homodata.edge_index[0,].max().item()+1)
print('the edge index looks like this:\n',homodata.edge_index[:,:81763])

unique_values, counts = torch.unique(homodata.node_type, return_counts=True) # count the number of diffrent nodes
print('There are two types of nodes:\n', unique_values, counts)  # here the user id starts at 0  and movie id starts at 9742
print('The edge index looks like this:\n',homodata.edge_index)

the total number of node is: 10352
the edge index looks like this:
 tensor([[    0,     0,     0,  ...,   609,   609,   609],
        [  610,   612,   615,  ..., 10072, 10073, 10113]], device='cuda:0')
There are two types of nodes:
 tensor([0, 1], device='cuda:0') tensor([ 610, 9742], device='cuda:0')
The edge index looks like this:
 tensor([[    0,     0,     0,  ..., 10072, 10073, 10113],
        [  610,   612,   615,  ...,   609,   609,   609]], device='cuda:0')


## Model Preparation
This model is almost the same as the LightGCN model in PyG : https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/models/lightgcn.html#LightGCN.

If you don't know the LightGCN model I would highly recommend you to read the paper: ***LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation*** and go through the example in Pyg: https://github.com/pyg-team/pytorch_geometric/blob/master/examples/lightgcn.py

However, PyG's LightGCN model doen't support network with node features, so we have to wirte a model ourself for this task.

$L_{\text{BPR}} = - \sum_{u=1}^{M} \sum_{i \in \mathcal{N}_u}\sum_{j \not\in \mathcal{N}_u} \ln \sigma(\hat{y}_{ui} - \hat{y}_{uj})+ \lambda \vert\vert \textbf{x}^{(0)} \vert\vert^2$

In [15]:
from torch_geometric.nn.models import LightGCN
from torch_geometric.nn.models.lightgcn import BPRLoss
from typing import Optional, Union

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn import Embedding, ModuleList, Linear
from torch.nn.modules.loss import _Loss

from torch_geometric.nn.conv import LGConv
from torch_geometric.typing import Adj, OptTensor
from torch_geometric.utils import is_sparse, to_edge_index


# PyG provides the LightGCN model, however this model doen't satisfy our needs as we have different node features
# So we need to overwrite the LightGCN model. The only difference is that we need to add a linear layer to transform the movie features to proper embedding
# And we also need to pass the node features to some functions
class CustomizedLightGCN(torch.nn.Module):
  r"""
  class:`~torch_geometric.nn.models.LightGCN` learns embeddings by linearly
  propagating them on the underlying graph, and uses the weighted sum of the
  embeddings learned at all layers as the final embedding


  Args:
      num_nodes (int): The number of nodes in the graph.
      embedding_dim (int): The dimensionality of node embeddings.
      num_item_features: The dimension of item features.
      num_layers (int): The number of
          :class:`~torch_geometric.nn.conv.LGConv` layers.
      alpha (float or torch.Tensor, optional): The scalar or vector
          specifying the re-weighting coefficients for aggregating the final
          embedding. If set to :obj:`None`, the uniform initialization of
          :obj:`1 / (num_layers + 1)` is used. (default: :obj:`None`)
      **kwargs (optional): Additional arguments of the underlying
          :class:`~torch_geometric.nn.conv.LGConv` layers.
    """
  def __init__(
        self,
        num_users: int,
        num_items: int,
        embedding_dim: int,
        num_item_features: int,
        num_layers: int,
        alpha: Optional[Union[float, Tensor]] = None,
        **kwargs,
    ):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.num_nodes = self.num_users + self.num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        if alpha is None:
            alpha = 1. / (num_layers + 1)

        if isinstance(alpha, Tensor):
            assert alpha.size(0) == num_layers + 1
        else:
            alpha = torch.tensor([alpha] * (num_layers + 1))
        self.register_buffer('alpha', alpha)

        self.user_embedding = Embedding(num_users, embedding_dim) # get user embedding as there is no user feature
        self.item_embedding = ModuleList([Linear(num_item_features,embedding_dim),
                                          torch.nn.BatchNorm1d(embedding_dim),
                                          Linear(embedding_dim,embedding_dim)
                                          ]) # get item embedding by transfer the 404 features to embedding_dim with a Linear Layer, you can design your own network here


        self.convs = ModuleList([LGConv(**kwargs) for _ in range(num_layers)])

        self.reset_parameters()

  def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        torch.nn.init.xavier_uniform_(self.user_embedding.weight)
        for layer in self.item_embedding:
          if isinstance(layer, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(layer.weight)
        for conv in self.convs:
            conv.reset_parameters()

  def get_embedding(
        self,
        edge_index: Adj,
        item_features: Tensor,
        edge_weight: OptTensor = None,
    ) -> Tensor:
        r"""Returns the embedding of nodes in the graph."""
        x1 = self.user_embedding.weight
        x2 = self.item_embedding[0](item_features)
        for layer in self.item_embedding[1:]:
          x2 = layer(x2) # transfer item features to a lower dimension

        x = torch.cat([x1,x2],dim=0)
        out = x * self.alpha[0] # equal weight

        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, edge_weight)
            out = out + x * self.alpha[i + 1]

        return out

  def forward(
        self,
        edge_index: Adj,
        item_features: Tensor,
        edge_label_index: OptTensor = None,
        edge_weight: OptTensor = None,
    ) -> Tensor:
        r"""Computes rankings for pairs of nodes.

        Args:
            edge_index (torch.Tensor or SparseTensor): Edge tensor specifying
                the connectivity of the graph.
            edge_label_index (torch.Tensor, optional): Edge tensor specifying
                the node pairs for which to compute rankings or probabilities.
                If :obj:`edge_label_index` is set to :obj:`None`, all edges in
                :obj:`edge_index` will be used instead. (default: :obj:`None`)
            item_features (torch.Tensor): The item feature tensor
            edge_weight (torch.Tensor, optional): The weight of each edge in
                :obj:`edge_index`. (default: :obj:`None`)
        """
        if edge_label_index is None:
            if is_sparse(edge_index):
                edge_label_index, _ = to_edge_index(edge_index)
            else:
                edge_label_index = edge_index

        out = self.get_embedding(edge_index,  item_features, edge_weight)

        out_src = out[edge_label_index[0]] # n*(embeding dim)
        out_dst = out[edge_label_index[1]]

        return (out_src * out_dst).sum(dim=-1) #n*1


  def recommend(
        self,
        edge_index: Adj,
        item_features: Tensor,
        edge_weight: OptTensor = None,
        src_index: OptTensor = None,
        dst_index: OptTensor = None,
        k: int = 1,
        sorted: bool = True,
    ) -> Tensor:
        r"""Get top-:math:`k` recommendations for nodes in :obj:`src_index`.

        Args:
            edge_index (torch.Tensor or SparseTensor): Edge tensor specifying
                the connectivity of the graph.
            item_features (torch.Tensor): The item feature tensor
            edge_weight (torch.Tensor, optional): The weight of each edge in
                :obj:`edge_index`. (default: :obj:`None`)
            src_index (torch.Tensor, optional): Node indices for which
                recommendations should be generated.
                If set to :obj:`None`, all nodes will be used.
                (default: :obj:`None`)
            dst_index (torch.Tensor, optional): Node indices which represent
                the possible recommendation choices.
                If set to :obj:`None`, all nodes will be used.
                (default: :obj:`None`)
            k (int, optional): Number of recommendations. (default: :obj:`1`)
            sorted (bool, optional): Whether to sort the recommendations
                by score. (default: :obj:`True`)
        """
        out_src = out_dst = self.get_embedding(edge_index, item_features, edge_weight)

        if src_index is not None:
            out_src = out_src[src_index]

        if dst_index is not None:
            out_dst = out_dst[dst_index]

        pred = out_src @ out_dst.t()
        top_index = pred.topk(k, dim=-1, sorted=sorted).indices

        if dst_index is not None:  # Map local top-indices to original indices.
            top_index = dst_index[top_index.view(-1)].view(*top_index.size())

        return top_index


  def recommendation_loss(
        self,
        pos_edge_rank: Tensor,
        neg_edge_rank: Tensor,
        item_features: Tensor,
        node_id: Optional[Tensor] = None,
        lambda_reg: float = 1e-4,
        **kwargs,
    ) -> Tensor:
        r"""Computes the model loss for a ranking objective via the Bayesian
        Personalized Ranking (BPR) loss.

        .. note::

            The i-th entry in the :obj:`pos_edge_rank` vector and i-th entry
            in the :obj:`neg_edge_rank` entry must correspond to ranks of
            positive and negative edges of the same entity (*e.g.*, user).

        Args:
            pos_edge_rank (torch.Tensor): Positive edge rankings.
            neg_edge_rank (torch.Tensor): Negative edge rankings.
            item_features (torch.Tensor): The item feature tensor
            node_id (torch.Tensor): The indices of the nodes involved for
                deriving a prediction for both positive and negative edges.
                If set to :obj:`None`, all nodes will be used.
            lambda_reg (int, optional): The :math:`L_2` regularization strength
                of the Bayesian Personalized Ranking (BPR) loss.
                (default: :obj:`1e-4`)
            **kwargs (optional): Additional arguments of the underlying
                :class:`torch_geometric.nn.models.lightgcn.BPRLoss` loss
                function.
        """
        loss_fn = BPRLoss(lambda_reg, **kwargs)
        x1 = self.user_embedding.weight

        x2 = self.item_embedding[0](item_features)
        for layer in self.item_embedding[1:]:
          x2 = layer(x2)

        emb = torch.cat([x1,x2],dim=0) # the embedding at the initial layer
        emb = emb if node_id is None else emb[node_id]
        return loss_fn(pos_edge_rank, neg_edge_rank, emb) # the emb is initial embedding that should passed to loss function as regularization


bpr_loss = BPRLoss()  # This is a unsupervised training, we feed the positive paris and negative pairs to the loss function


## Build the model

In [16]:
num_users, num_items = data['user'].num_nodes, data['movie'].num_nodes # 610 users and 9742 movies
embedding_dim = 64
num_item_features = data['movie'].x.shape[1] # 404 features
num_layers = 3

model = CustomizedLightGCN(num_users = num_users,
                num_items = num_items,
                embedding_dim = embedding_dim,
                num_item_features = num_item_features,
                num_layers = num_layers).to(device)

In [17]:
item_features = data['movie'].x.to(device)

In [18]:
from torch_geometric.nn import summary
data['movie'].x.to(device)
print(summary(model,homodata.edge_index,item_features))

+------------------------------+--------------------------+----------------+----------+
| Layer                        | Input Shape              | Output Shape   | #Param   |
|------------------------------+--------------------------+----------------+----------|
| CustomizedLightGCN           | [2, 163526], [9742, 404] | [163526]       | 69,248   |
| ├─(user_embedding)Embedding  | --                       | --             | 39,040   |
| ├─(item_embedding)ModuleList | --                       | --             | 30,208   |
| │    └─(0)Linear             | [9742, 404]              | [9742, 64]     | 25,920   |
| │    └─(1)BatchNorm1d        | [9742, 64]               | [9742, 64]     | 128      |
| │    └─(2)Linear             | [9742, 64]               | [9742, 64]     | 4,160    |
| ├─(convs)ModuleList          | --                       | --             | --       |
| │    └─(0)LGConv             | [10352, 64], [2, 163526] | [10352, 64]    | --       |
| │    └─(1)LGConv             |

## Model training and Test


##### split the data into training and evaluation set
First, we splite the edges to training and testing. Recall that in semi-supervised learning, we hide some edges from model during training, and use the hidden edges for testing

In [19]:
unique_values, counts = torch.unique(homodata.node_type, return_counts=True) # count the number of diffrent nodes
print('There are two types of nodes:\n', unique_values, counts)
# recall that in our network, the user id starts at 0 and movie_id starts at 9742

There are two types of nodes:
 tensor([0, 1], device='cuda:0') tensor([ 610, 9742], device='cuda:0')


In [20]:
from sklearn.model_selection import train_test_split
num_undirected_edge = int(homodata['edge_index'].shape[1]/2)
indices = range(num_undirected_edge) # the indices from items to users

# Split the indices into train and test sets
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_edge_index = torch.cat([homodata['edge_index'][:, train_indices], homodata['edge_index'][:, torch.tensor(train_indices) + num_undirected_edge]],dim=1) # this will be the basic edge_index for training

test_edge_index_label = torch.cat([homodata['edge_index'][:, test_indices], homodata['edge_index'][:, torch.tensor(test_indices) + num_undirected_edge]],dim=1) # this is the label edge_index for evaluation, won't be seen in training


##### prepare the basic setting and data loader
During training,  we take a batch_size of edge_index a time, in this case, 200 edges as positive edges, and sample another 200 edges, which don't exist, as negative edges

In [21]:

mask = train_edge_index[0] < train_edge_index[1] # filter edges from user to items

train_edge_index_label = train_edge_index[:, mask] # user to item edge_list, this will be the label index for training

batch_size = 50

train_loader = torch.utils.data.DataLoader(
    range(train_edge_index_label.size(1)), # edge number
    shuffle=True,
    batch_size=batch_size,
) #

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

##### train

In [22]:
# for every epoch there is a train
data['movie'].x.to(device)

from torch_geometric.utils import structured_negative_sampling

def train():
    model.train()
    total_loss = total_examples = 0

    for index in tqdm(train_loader):
        # Sample positive and negative labels.
        pos_edge_label_index = train_edge_index_label[:, index] # user to item edges

        _,_,neg = structured_negative_sampling(train_edge_index_label,) # for a given edge_index, sample a negative edge
        neg_edge_label_index  = torch.stack([pos_edge_label_index[0],neg[index]], dim=0).to(device)


        edge_label_index = torch.cat([
            pos_edge_label_index, # shape [2,batch_size]
            neg_edge_label_index, # shape [2,batch_size]
        ], dim=1) #[2,2*batch_size]

        optimizer.zero_grad()
        pos_rank, neg_rank = model(train_edge_index, item_features, edge_label_index,).chunk(2) # be careful of the difference between the train_edge_index and edge_label_index ,
        # thrid argument is edge_label_index, specifying the node pairs for which to compute rankings or probabilities.
        # the chunk method makes the result equally split into two chunks, i.e. two views of tensor

        # the rank is just a elmentisie multiply of the final node embeddings

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
            item_features = item_features,
        )
        # the loss is defined as log_prob = F.logsigmoid(positives - negatives).mean(),  thus it is the average loss of all source nodes
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()# the (index.numel(), ) defines the size, .numel() means the total number of element in this tensor
        total_examples += pos_rank.numel()

    return total_loss / total_examples


##### evaluation

In [23]:
from torch_geometric.utils import degree
def test(k: int):
    model.eval()
    emb = model.get_embedding(train_edge_index, item_features) # use the training data to get embedding
    user_emb, item_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0

    for start in range(0, num_users, batch_size):
        end = min(start + batch_size, num_users)

        logits = user_emb[start : end] @ item_emb.t() # logits is a [batch_size,item_num] matrix, the ranking matrix

        # Exclude training edges:
        # recall that we already take the edge in train_edge_index_label for training, so we don't want to use them for eavluation,
        # becasue those edges(or node pairs) theoritically have a high ranking score and will always be recommended, so we need to make them to -inf
        # with source node ranging from [0, batch_size], we want to find out the X edegs in the trainnig_edge_index satrting from the #batch_size nodes to any items

        mask = ((train_edge_index_label[0] >= start) &
                (train_edge_index_label[0] < end)) # the mask indicates all the edges we have used for training, where the source node is included in this batch

        logits[train_edge_index_label[0, mask] - start, # train_edge_label_index[0, mask] is a tensor with value starting between 'start' and 'end',
               train_edge_index_label[1, mask] - num_users ] = float('-inf') #



        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((test_edge_index_label[0] >= start) &
                (test_edge_index_label[0] < end))

        ground_truth[test_edge_index_label[0, mask] - start,
                     test_edge_index_label[1, mask] - num_users] = True # so long if an edge exists in the test edge index,
                                                  # the corresponding node paris should be True otherwise False
        node_count = degree(test_edge_index_label[0, mask] - start,
                            num_nodes=logits.size(0)) # count the number of degree for each node in test set

        topk_index = logits.topk(k, dim=-1).indices # for each user, get the top k recommended item indices, shape[num_users,k]

        isin_mat = ground_truth.gather(dim=1, index=topk_index) # fetch the value according to the topk_index on the first dimension
        # this will be a N*k shape matrix, with each element bool
        # SEE REF : https://pytorch.org/docs/stable/generated/torch.gather.html

        precision += float((isin_mat.sum(dim=-1) / k).sum()) # precision at k
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum()) # recall at k
        total_examples += int((node_count > 0).sum())

    return precision / total_examples, recall / total_examples # the avearge precision and recall

##### result

In [25]:
for epoch in range(1, 11): # I would recommed 10 epochs to start with, you can train more epoches if you want
    loss = train()
    precision, recall = test(k=10)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Precision@10: '
          f'{precision:.4f}, Recall@10: {recall:.4f}')

  0%|          | 0/1309 [00:00<?, ?it/s]

Epoch: 001, Loss: 0.2982, Precision@10: 0.2031, Recall@10: 0.1233


  0%|          | 0/1309 [00:00<?, ?it/s]

Epoch: 002, Loss: 0.2934, Precision@10: 0.2117, Recall@10: 0.1276


  0%|          | 0/1309 [00:00<?, ?it/s]

...

Epoch: 008, Loss: 0.2566, Precision@10: 0.2158, Recall@10: 0.1262


  0%|          | 0/1309 [00:00<?, ?it/s]

Epoch: 009, Loss: 0.2539, Precision@10: 0.2211, Recall@10: 0.1317


  0%|          | 0/1309 [00:00<?, ?it/s]

Epoch: 010, Loss: 0.2509, Precision@10: 0.2294, Recall@10: 0.1354


##### let's make some recommendation

In [26]:
src_indices = torch.tensor([0]).to(device)
dst_indices = torch.arange(num_users, num_users+num_items).to(device)
model.recommend(homodata.edge_index, item_features, src_index=src_indices, dst_index=dst_indices, k=10)  # recommend 10 movies for user 0



tensor([[1028, 1510, 1119, 1116,  834, 1120,  610, 1508, 1156,  867]],
       device='cuda:0')

In [ ]:
# compare the result with actual neighbor node

In [ ]:
homodata.edge_index[1, homodata.edge_index[0] == 0]

tensor([ 610,  612,  615,  653,  656,  672,  699,  707,  734,  740,  746,  794,
         800,  807,  811,  834,  867,  885,  901,  917,  924,  930,  935,  977,
         994, 1008, 1028, 1046, 1071, 1086, 1094, 1095, 1118, 1119, 1120, 1123,
        1130, 1156, 1161, 1169, 1202, 1204, 1225, 1242, 1311, 1315, 1330, 1333,
        1344, 1377, 1391, 1392, 1393, 1396, 1397, 1398, 1399, 1407, 1411, 1420,
        1425, 1430, 1438, 1439, 1441, 1446, 1466, 1473, 1508, 1509, 1510, 1517,
        1519, 1521, 1524, 1525, 1531, 1533, 1535, 1537, 1549, 1565, 1567, 1574,
        1579, 1584, 1587, 1591, 1600, 1606, 1646, 1670, 1686, 1694, 1720, 1736,
        1756, 1764, 1781, 1791, 1793, 1800, 1828, 1830, 1834, 1845, 1871, 1908,
        1929, 1936, 1942, 1943, 2011, 2017, 2041, 2054, 2085, 2090, 2097, 2103,
        2113, 2115, 2126, 2127, 2132, 2136, 2153, 2163, 2166, 2167, 2169, 2172,
        2177, 2186, 2187, 2205, 2207, 2209, 2211, 2227, 2238, 2254, 2297, 2301,
        2314, 2344, 2365, 2378, 2398, 24